In [47]:
# Imports
import cv2
import os
import numpy as np
import logging
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm
from contextlib import contextmanager

# Logging Setup
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [48]:
# Configurations
class Config:
    PREVIEW_ONLY = False  # Flag für Vorschau
    BACKGROUND_COLOR = '#ADD8E6'  # Hellblau

class VideoConfig:
    BACKGROUND_VIDEO_PATH = "background.mp4"
    OUTPUT_VIDEO_PATH = "quote_video.mp4"
    FPS = 20.0
    #FOURCC = "mp4v"
    FOURCC = "avc1"    
    FONT_SIZE = 40
    TEXT_COLOR = (255, 255, 255)  # White
    FONT_PATH = "noggin_bulb.ttf"

class TextConfig:
    # Text Inhalte mit Timing
    DEFAULT_TEXTS = {
        'quote': {
            'text': "What motivates you \n the most?",
            'start_frame': 0,
            'end_frame': None
        },
        'option_a': {
            'text': "a) Success",
            'start_frame': 30,
            'end_frame': None
        },
        'option_b': {
            'text': "b) Freedom",
            'start_frame': 60,
            'end_frame': None
        },
        'option_c': {
            'text': "c) Learning",
            'start_frame': 90,
            'end_frame': None
        },
        'option_d': {
            'text': "d) Creating",
            'start_frame': 120,
            'end_frame': None
        }
    }
    
    # Positionen bleiben gleich
    POSITIONS = {
        'quote':  2.1,
        'option_a':  1.85,
        'option_b': 1.75,
        'option_c': 1.65,
        'option_d': 1.55
    }
    
    # Schriftgrößen bleiben gleich
    FONT_SIZES = {
        'quote': 55,    # Größere Überschrift
        'option_a': 40, # Kleinere Optionen
        'option_b': 40,
        'option_c': 40,
        'option_d': 40
    }

In [49]:
# Context Manager für Video-Ressourcen
@contextmanager
def video_manager(cap, out):
    try:
        yield cap, out
    finally:
        if cap is not None:
            cap.release()
        if out is not None:
            out.release()
        cv2.destroyAllWindows()

In [50]:
def validate_params(width: int, height: int, fontSize: int) -> bool:
    """Validiert die Input-Parameter"""
    if width <= 0 or height <= 0:
        logging.error("Ungültige Bildgröße")
        return False
    if fontSize <= 0:
        logging.error("Ungültige Schriftgröße")
        return False
    return True

In [51]:
def initializeVideo(input_path: str) -> tuple:
    """Initialisiert Video Capture und Writer"""
    try:
        if not os.path.exists(input_path):
            logging.error(f"Video nicht gefunden: {input_path}")
            return None, None, None, None
            
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            logging.error(f"Video konnte nicht geöffnet werden: {input_path}")
            return None, None, None, None
        
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        if not validate_params(width, height, VideoConfig.FONT_SIZE):
            cap.release()
            return None, None, None, None
        
        fourcc = cv2.VideoWriter_fourcc(*VideoConfig.FOURCC)
        out = cv2.VideoWriter(
            VideoConfig.OUTPUT_VIDEO_PATH, 
            fourcc, 
            VideoConfig.FPS, 
            (width, height)
        )
        
        if not out.isOpened():
            logging.error("Output Video konnte nicht erstellt werden")
            cap.release()
            return None, None, None, None
            
        return cap, out, width, height
        
    except Exception as e:
        logging.error(f"Ein unerwarteter Fehler ist aufgetreten: {e}")
        if 'cap' in locals():
            cap.release()
        return None, None, None, None

In [52]:
def createImage(width, height, bgColor=(0,0,0,0), fontPath="arial.ttf"):
    text_image = Image.new("RGBA", (width, height), bgColor)
    draw = ImageDraw.Draw(text_image)
    
    # Dictionary für verschiedene Font-Größen
    fonts = {}
    for size in set(TextConfig.FONT_SIZES.values()):
        try:
            fonts[size] = ImageFont.truetype(fontPath, size)
        except:
            print(f"Warnung: Font {fontPath} nicht gefunden, nutze Standard-Font")
            fonts[size] = ImageFont.load_default()
            
    return text_image, draw, fonts

In [53]:
def addTexts(texts, positions, draw, font, width, height, text_color='white'):
    # Schriftgrößen-Dictionary
    font_sizes = {
        'quote': 55,    # Überschrift größer
        'option_a': 40, # Optionen kleiner
        'option_b': 40,
        'option_c': 40,
        'option_d': 40
    }
    
    shadow_color = '#333333'
    blur_amount = 3
    offset = 3
    
    for key, text in texts.items():
        try:
            # Wähle die richtige Schriftgröße für jeden Text
            font = ImageFont.truetype("arial.ttf", font_sizes[key])
        except:
            print(f"Warnung: Font nicht gefunden, nutze Standard-Font")
            font = ImageFont.load_default()
            
        x = width // 2
        y = height // positions[key]
        
        # Blur-Schatten
        for blur in range(blur_amount):
            draw.text(
                (x + offset + blur, y + offset + blur),
                text,
                font=font,
                anchor="mm",
                fill=shadow_color
            )
        
        # Haupttext
        draw.text(
            (x, y),
            text,
            font=font,
            anchor="mm",
            fill=text_color
        )

In [54]:
def processVideo(cap, out, text_image, font) -> bool:
    """Verarbeitet das Video und fügt Text-Overlay hinzu"""
    try:
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        current_frame = 0
        
        with tqdm(total=total_frames, desc="Processing video") as pbar:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Erstelle ein leeres Text-Image für diesen Frame
                frame_text = Image.new('RGBA', text_image.size, (0,0,0,0))
                draw = ImageDraw.Draw(frame_text)
                
                # Füge nur die Texte hinzu, die in diesem Frame sichtbar sein sollen
                for key, text_info in TextConfig.DEFAULT_TEXTS.items():
                    start = text_info['start_frame'] or 0
                    end = text_info['end_frame'] or total_frames
                    
                    if start <= current_frame <= end:
                        # Verwende die richtigen Positionen und Schriftgrößen
                        position = TextConfig.POSITIONS[key]
                        font_size = TextConfig.FONT_SIZES[key]
                        current_font = ImageFont.truetype("arial.ttf", font_size)
                        
                        draw.text(
                            (text_image.width // 2, text_image.height // position),
                            text_info['text'],
                            font=current_font,
                            anchor="mm",
                            fill='white'
                        )
                
                # Frame verarbeiten
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
                frame_pil = Image.fromarray(frame)
                frame_pil.paste(frame_text, mask=frame_text)
                frame_array = np.array(frame_pil)
                frame = cv2.cvtColor(frame_array, cv2.COLOR_RGBA2BGR)
                out.write(frame)
                
                current_frame += 1
                pbar.update(1)
                
        return True
        
    except Exception as e:
        logging.error(f"Fehler beim Video-Processing: {e}")
        return False

In [55]:
from IPython.display import Video, display
def showVideo():
    # Nach der Video-Erstellung:
    print("Video wird angezeigt...")
    display(Video(VideoConfig.OUTPUT_VIDEO_PATH, embed=True, width=400))

In [56]:
from IPython.display import HTML
def showVideoAlternative():
    HTML("""
    <video width="400" controls>
        <source src="quote_video.mp4" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    """)

In [57]:
from IPython.display import Javascript
def show_popup():
    js = """
    alert('Notebook Ausführung abgeschlossen!');
    """
    display(Javascript(js))

In [58]:
import os

def openVideoInPlayer():
    if os.name == 'nt':  # Windows
        os.system(f'start quote_video.mp4')
    elif os.name == 'posix':  # Mac/Linux
        os.system(f'open quote_video.mp4')

In [ ]:
def check_video():
    cap = cv2.VideoCapture("quote_video.mp4")
    
    if not cap.isOpened():
        print("Fehler: Video kann nicht geöffnet werden")
        return
        
    # Eigenschaften auslesen
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    codec = int(cap.get(cv2.CAP_PROP_FOURCC))
    
    print(f"Video Eigenschaften:")
    print(f"Auflösung: {width}x{height}")
    print(f"FPS: {fps}")
    print(f"Anzahl Frames: {frame_count}")
    print(f"Codec: {codec}")
    
    # Ressourcen freigeben
    cap.release()

check_video()


In [60]:
def main():
    try:
        if Config.PREVIEW_ONLY:
            # Nur Vorschau erstellen
            width, height = 1080, 1920
            text_image, draw, font = createImage(width, height, Config.BACKGROUND_COLOR)
            
            # Anpassen für neue Textstruktur
            texts = {k: v['text'] for k, v in TextConfig.DEFAULT_TEXTS.items()}
            
            addTexts(texts, TextConfig.POSITIONS, 
                    draw, font, width, height)
            text_image.show()
            show_popup()
            return
            
        # Video initialisieren
        cap, out, width, height = initializeVideo(VideoConfig.BACKGROUND_VIDEO_PATH)
        if cap is None:
            return
            
        with video_manager(cap, out):
            text_image, draw, font = createImage(width, height)
            # Hier auch anpassen
            texts = {k: v['text'] for k, v in TextConfig.DEFAULT_TEXTS.items()}
            addTexts(texts, TextConfig.POSITIONS, 
                    draw, font, width, height)
            
            if processVideo(cap, out, text_image, font):
                logging.info("Video erfolgreich erstellt!")
                showVideo()
                show_popup()
            else:
                logging.error("Fehler bei der Video-Erstellung")
                
    except Exception as e:
        logging.error(f"Ein Fehler ist aufgetreten: {e}")

In [ ]:
# Ausführen der Hauptfunktion
main()
openVideoInPlayer()